In [16]:
"""
XGBoost multiclassifier for roof materials in D.C.
"""

import os, sys, glob, time
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray as rxr
import rasterio as rio
import xgboost as xgb
import optuna
import gc

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, accuracy_score

# Functions script import
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

print(os.getcwd())

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping'
# homedir = '/home/jovyan/data-store/data/iplant/home/maco4303/data/' # cyverse

print("Imports successful!")

/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/code
Imports successful!


In [2]:
results_dir = os.path.join(maindir, 'results/xgboost-cv/')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
print(f"Saving results to {results_dir}")

Saving results to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping/results/xgboost-cv/


In [3]:
# Load the image data

In [4]:
# Load our image data to check on the format
stack_da_fp = os.path.join(maindir,'data/spatial/mod/dc_data/planet-data/dc_0623_psscene8b_final_norm.tif')
# stack_da_fp = os.path.join(homedir,'OPP/planet/dc/dc_0623_psscene8b_final_norm.tif') # cyverse
stack_da = rxr.open_rasterio(stack_da_fp, masked=True, cache=False).squeeze()
print_raster(stack_da, open_file=False)
band_names = list(stack_da.long_name)
print(f"Band names: {band_names}")
del stack_da

shape: (8223, 6714)
resolution: (3.0, -3.0)
bounds: (315267.0, 4294629.0, 335409.0, 4319298.0)
sum: 8.181640625
CRS: EPSG:32618
NoData: None
Band names: ['nir', 'NDBIbg', 'NDBIrg', 'NISI', 'MNF1', 'NISI5x5']


In [5]:
# Load the reference data

In [6]:
# Load the training data (footprints)
ref_fp = os.path.join(maindir,'data/spatial/mod/dc_data/training/dc_data_reference_footprints_sampled.gpkg')
# ref_fp = os.path.join(homedir,'OPP/training/dc/dc_data_reference_footprints_sampled.gpkg')
ref = gpd.read_file(ref_fp)
ref.head()

,uid,class_code,description,areaUTMsqft,lotSizeSqft,nir,NDBIbg,NDBIrg,NISI,MNF1,NISI5x5,geometry
0,1CS,CS,Composition Shingle,357.783709,5574.0,-0.246291,0.373826,0.472331,0.204085,0.080244,2.598158,"POLYGON ((324215.868 4313568.665, 324215.792 4..."
1,3CS,CS,Composition Shingle,918.640862,1111.0,-0.856021,0.632535,1.064584,1.001515,0.569262,0.367447,"POLYGON ((324602.816 4311717.247, 324604.322 4..."
2,9CS,CS,Composition Shingle,1383.414170,4222.0,-0.864054,0.813552,0.806408,0.734980,0.218044,-0.683204,"POLYGON ((327253.581 4300371.859, 327258.154 4..."
3,19CS,CS,Composition Shingle,836.410297,3450.0,-0.735984,0.305262,0.856115,0.508690,0.070037,0.680645,"POLYGON ((333608.13 4306267.691, 333607.957 43..."
4,27CS,CS,Composition Shingle,330.514264,4352.0,-0.418819,0.516599,0.897064,0.448048,0.185206,-0.394834,"POLYGON ((326482.699 4300939.466, 326487.386 4..."


In [7]:
print(ref['class_code'].value_counts())

class_code
CS    37133
ME    36864
SL    15267
UR     1279
WS     1152
TL      924
SH      783
Name: count, dtype: int64


In [8]:
# Create a numeric code column
ref['code'], _ = pd.factorize(ref['class_code'])

# Create a dictionary mapping class_code to code
code_mapping = dict(zip(ref['class_code'], ref['code']))
desc_mapping = dict(zip(ref['class_code'], ref['description']))

print(f'Code map: \n{code_mapping}\nDescription map: \n{desc_mapping}')

Code map: 
{'CS': 0, 'ME': 1, 'SL': 2, 'UR': 3, 'TL': 4, 'WS': 5, 'SH': 6}
Description map: 
{'CS': 'Composition Shingle', 'ME': 'Metal', 'SL': 'Slate', 'UR': 'Urethane', 'TL': 'Tile', 'WS': 'Wood shake/shingle', 'SH': 'Shingle'}


In [9]:
# Create a holdout sample for model evaluation later on

In [10]:
# Create the holdout data (independent from model training in CV)
holdout_fraction = 0.20 # excluding 20% of the data for holdout/independent testing
train_df, holdout_df, _ = split_training_data(ref, ts=holdout_fraction, vs=None) # no validation needed yet

print("Training set class distribution:\n", train_df[['class_code','code']].value_counts())
print("Holdout set class distribution:\n", holdout_df[['class_code','code']].value_counts())

Training set class distribution:
 class_code  code
CS          0       29706
ME          1       29491
SL          2       12213
UR          3        1023
WS          5         921
TL          4         739
SH          6         626
Name: count, dtype: int64
Holdout set class distribution:
 class_code  code
CS          0       7427
ME          1       7373
SL          2       3054
UR          3        256
WS          5        231
TL          4        185
SH          6        157
Name: count, dtype: int64


In [11]:
# Parameter tuning using Optuna

In [17]:
# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(train_df['code'])),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-6, 1e-2),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 101, 1001),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-9, 1.0),
        'random_state': 44,
    }

    # Stratified 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    f1_scores = [] # to store the validation F1 for each fold
    
    for train_idx, val_idx in skf.split(train_df, train_df['class_code']):
        # Extract the train and validation sets
        fold_train = train_df.iloc[train_idx]
        fold_val = train_df.iloc[val_idx]

        # Undersample majority class in training set only
        fold_train_bal = balance_sampling(fold_train, ratio=5, strategy='undersample')

        # Calculate the class weights
        val_counts = fold_train_bal['code'].value_counts()
        total_samples = sum(val_counts)
        class_weights = np.array([total_samples / count for count in val_counts])
        class_weights_n = class_weights / class_weights.sum() # normalize the weights
        class_weights_n = {cls: weight for cls, weight in zip(np.unique(fold_train_bal['code']), class_weights)}

        # Define our train/validation sets
        X_train = fold_train_bal[band_names].astype(float)
        y_train = fold_train_bal['code'].astype(int)
        X_val = fold_val[band_names].astype(float)
        y_val = fold_val['code'].astype(int)
        
        del fold_train, fold_val, fold_train_bal # clear up space

        # Fit the XGBoost with params
        xgb_model = xgb.XGBClassifier(n_jobs=1, **params) # avoid over-using CPU resources
        xgb_model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],  # Specify validation set
            sample_weight=y_train.map(class_weights_n).values,
            eval_metric=['mlogloss'], # good for multiclass
            early_stopping_rounds=10, # stop if no improvement
            verbose=False
        )

        # Evaluate F1-score on the validation set
        y_pred = xgb_model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='weighted')
        f1_scores.append(f1)

        del y_pred, f1, X_train, y_train, X_val, y_val, xgb_model

    # Calculate mean metrics across folds
    f1_ = np.mean(f1_scores)

    del f1_scores
    gc.collect()

    return f1_  # Optuna will maximize this score

print("Optuna function ready to go !!!")

Optuna function ready to go !!!


In [18]:
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
# Implement the optuna objective study
study = optuna.create_study(direction='maximize', study_name='dc_xgboost_tuning') # maximize the scoring criteria
study.optimize(objective, n_trials=50, n_jobs=5, show_progress_bar=True)  # Adjust the number of trials as needed
print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters found: {best_params}")
# Save the Optuna study for later analysis
study.trials_dataframe().to_csv(os.path.join(results_dir, 'dc-xgboost-cv_optuna_trials.csv'), index=False)

[I 2024-09-24 14:46:30,769] A new study created in memory with name: no-name-cfe30995-3e10-49b8-8e9a-266a57eabb9c


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



Best trial: 2. Best value: 0.477633:   2%|▏        | 1/50 [01:28<1:12:19, 88.57s/it]

[I 2024-09-24 14:47:59,343] Trial 2 finished with value: 0.47763309059404513 and parameters: {'learning_rate': 0.00025230082034872765, 'max_depth': 5, 'n_estimators': 455, 'min_child_weight': 5, 'subsample': 0.525296899263471, 'colsample_bytree': 0.8701283142284606, 'gamma': 1.4687384197120863e-06}. Best is trial 2 with value: 0.47763309059404513.


Best trial: 4. Best value: 0.570337:   4%|▍          | 2/50 [01:57<42:41, 53.37s/it]

...

[I 2024-09-24 15:34:03,556] Trial 47 finished with value: 0.5234573145503971 and parameters: {'learning_rate': 0.00271650359022917, 'max_depth': 7, 'n_estimators': 926, 'min_child_weight': 1, 'subsample': 0.5808423423889431, 'colsample_bytree': 0.7603459202803521, 'gamma': 0.01801747679816116}. Best is trial 29 with value: 0.6155980935998727.


Best trial: 29. Best value: 0.615598:  98%|████████▊| 49/50 [49:14<01:01, 61.08s/it]

[I 2024-09-24 15:35:45,388] Trial 46 finished with value: 0.5549872150635996 and parameters: {'learning_rate': 0.0019323413493062568, 'max_depth': 9, 'n_estimators': 928, 'min_child_weight': 1, 'subsample': 0.5806025829791039, 'colsample_bytree': 0.878637434726981, 'gamma': 0.021038079988934283}. Best is trial 29 with value: 0.6155980935998727.


Best trial: 29. Best value: 0.615598: 100%|█████████| 50/50 [50:08<00:00, 60.18s/it]

[I 2024-09-24 15:36:39,749] Trial 49 finished with value: 0.5676420141271533 and parameters: {'learning_rate': 0.004765351464785309, 'max_depth': 9, 'n_estimators': 905, 'min_child_weight': 1, 'subsample': 0.5774900710593927, 'colsample_bytree': 0.7649836120110108, 'gamma': 0.0006878710540806738}. Best is trial 29 with value: 0.6155980935998727.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Best hyperparameters found: {'learning_rate': 1.1170160434715529e-05, 'max_depth': 14, 'n_estimators': 821, 'min_child_weight': 2, 'subsample': 0.5077057096262747, 'colsample_bytree': 0.8191150936146058, 'gamma': 1.997795558541556e-06}


In [24]:
best_params

{'learning_rate': 1.1170160434715529e-05,
 'max_depth': 14,
 'n_estimators': 821,
 'min_child_weight': 2,
 'subsample': 0.5077057096262747,
 'colsample_bytree': 0.8191150936146058,
 'gamma': 1.997795558541556e-06}

In [25]:
best_params = {
    'learning_rate': 1.1170160434715529e-05,
    'max_depth': 14,
    'n_estimators': 821,
    'min_child_weight': 2,
    'subsample': 0.5077057096262747,
    'colsample_bytree': 0.8191150936146058,
    'gamma': 1.997795558541556e-06
}

In [ ]:
# Model run w/ best parameters

In [26]:
# # Create a dictionary from "best_params"
# best_params_fp = os.path.join(results_dir, 'dc-xgboost-cv_optuna_trials.csv')
# best_params_df = pd.read_csv(best_params_fp)
# print(best_params_df.head())

In [ ]:
t0 = time.time()

# Define dataframes to store results for this feature set
results = pd.DataFrame()  # to store the model performance metrics

# Set up the stratified K-fold CV
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=44)

# Loop the folds
fold_idx = 1
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['class_code'])):
    t00 = time.time()
    
    print(f"Processing fold {fold_id + 1}/{n_folds}...")

    # Get the train and validation splits for this fold
    fold_train = train_df.iloc[train_idx]
    fold_val = train_df.iloc[val_idx]

    # Apply random undersampling to the training data within this fold
    fold_train_bal = balance_sampling(fold_train, ratio=5, strategy='undersample')
    # Display the class distribution in the undersampled training data for this fold
    print(f"Train class distribution (fold {fold_idx + 1}):\n", fold_train_bal['code'].value_counts().to_dict())
    print(f"Validation class distribution (fold {fold_idx + 1}):\n", fold_val['code'].value_counts().to_dict())
    del fold_train

    # Calculate the class weights
    val_counts = fold_train_bal['code'].value_counts()
    total_samples = sum(val_counts)
    class_weights = np.array([total_samples / count for count in val_counts])
    class_weights_n = class_weights / class_weights.sum() # normalize the weights
    class_weights_n = {cls: weight for cls, weight in zip(np.unique(fold_train_bal['code']), class_weights)}
    # print(f"Class weights: {class_weights_n}")
    del val_counts, total_samples, class_weights # clean up some

    # Initialize the XGBoost classifier for multi-class classification
    xgb_model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(np.unique(fold_train_bal['code'])),
        n_estimators=1001,
        learning_rate=0.01,
        max_depth=8,
        random_state=44,
        eval_metric=['mlogloss', 'rmse']
    )

    # Fit the model
    X_train = fold_train_bal[band_names].astype(float)
    y_train = fold_train_bal['code'].astype(int)
    X_val = fold_val[band_names].astype(float)
    y_val = fold_val['code'].astype(int)
    
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],  # Specify validation set
        sample_weight=y.map(class_weights_n).values,
        eval_metric=['mlogloss'], # good for multiclass
        early_stopping_rounds=10, # stop if no improvement
        verbose=False
    )

    # Retrieve the evaluation metrics for the fold
    eval_results = xgb_model.evals_result()
    print(f"Evaluation metrics for fold {fold_idx + 1}:", eval_results)

    # Predict on the validation set
    
    y_pred = xgb_model.predict(X_val)

    # Calculate the accuracy and f1-score (weighted)
    f1 = f1_score(y_val, y_pred, average='weighted')
    acc = accuracy_score(y_val, y_pred, average='weighted')

    # Store the metrics into the results data frame
    fold_results = pd.DataFrame({
        'fold': [fold_idx],
        'mlogloss': [eval_metrics['mlogloss']],
        'rmse': [eval_metrics['rmse']]
        'f1_val': [f1],
        'acc_val': [acc],
        'val_labs': [y_val],
        'val_preds': [y_pred]
    })
    # Concatenate with the results data frame
    results = pd.concat([results, fold_results], ignore_index=True)

    # Save the model for further evaluation
    model_filename = f"dc-xgb_fold{fold_idx + 1}.model"
    xgb_model.save_model(os.path.join(results_dir, model_fn))

    t1 = (time.time() - t00) / 60
    print(f"Total elapsed time for fold {fold_idx}: {t1:.2f} minutes.")

    del X, y, X_val, y_val, y_pred
    gc.collect()

    fold_idx += 1 # fold counter
    print("\n~~~~~~~~~~\n")

t2 = (time.time() - t0) / 60
print(f"Total elapsed time: {t2:.2f} minutes.")

In [ ]:
# # Append the feature set-specific results to the overall results dataframes
# all_results = pd.concat([all_results, results], ignore_index=True)
# all_feat_imps = pd.concat([all_feat_imps, feat_imps], ignore_index=True)
# all_prob_preds = pd.concat([all_prob_preds, prob_preds], ignore_index=True)

# del results, feat_imps, prob_preds

In [ ]:
    # Store feature importance
    fold_imps = pd.DataFrame({
        'Fold': fold_idx,
        'Feature': X.columns,
        'Importance': xgb_model.feature_importances_
    })
    # Append to the running data frame
    feat_imps = pd.concat([feat_imps, fold_imps], axis=0)
    
    # Store the probability values for cutoff testing
    y_pred_probs = xgb_model.predict_proba(X_test)

    # Store probabilities and true labels
    fold_probs = pd.DataFrame({
        'Fold': fold_idx,
        'TrueLabel': y_val,
        'PredictedProb': list(y_pred_probs)
    })

    # Concatenate the feature importances and probabilities
    prob_preds = pd.concat([prob_preds, fold_probs], ignore_index=True)

In [ ]:
results.head()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Initialize the confusion matrix accumulator
n_classes = len(np.unique(y))  # Number of classes
cm_accumulator = np.zeros((n_classes, n_classes))

all_true_labels = []
all_pred_labels = []

class_acc_reports = []

# Loop over the folds and compute confusion matrices
for fold in range(1, fold_idx):
    fold_data = prob_preds[prob_preds['Fold'] == fold]
    true_labels = fold_data['TrueLabel']
    pred_labels = np.argmax(np.vstack(fold_data['PredictedProb']), axis=1)

    all_true_labels.extend(true_labels)
    all_pred_labels.extend(pred_labels)

    # Compute the classification report for this fold
    cr_fold = classification_report(true_labels, pred_labels, target_names=cor_labels, output_dict=True)
    # Convert the classification report to a DataFrame and store it
    cr_df_fold = pd.DataFrame(cr_fold).transpose()
    class_acc_reports.append(cr_df_fold)
    
    # Compute the confusion matrix for this fold
    cm_fold = confusion_matrix(true_labels, pred_labels, labels=np.unique(y))

    # Accumulate the confusion matrix
    cm_accumulator += cm_fold

# Average the confusion matrix by the number of folds
cm_avg = cm_accumulator / (fold_idx - 1)

# Round to the nearest integer (optional, as you mentioned it's fine)
cm_avg = np.rint(cm_avg).astype(int)

# Display the averaged confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=np.unique(y))

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax)
plt.title('Averaged Confusion Matrix Across Folds')

# Save the figure
plt.savefig(os.path.join('/home/jovyan/data-store/results/FigX_xgboost_confusion_matrix_avg.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
cr_avg = pd.concat(class_acc_reports).groupby(level=0).mean()
print(cr_avg)

In [ ]:
from sklearn.metrics import classification_report

# Create a classification report and convert it to a DataFrame
cor_labels = [label for label, idx in sorted(class_mapping.items(), key=lambda item: item[1])]
cr_df = pd.DataFrame(classification_report(all_true_labels, all_pred_labels, target_names=cor_labels, output_dict=True)).transpose()

# Compute the average accuracy metrics across the 10 folds
average_metrics = cr_df.loc[cor_labels].mean()

# Display the DataFrame
cr_df

In [ ]:
# Save out the results
results_dir = '/home/jovyan/data-store/results/'
results.to_csv(os.path.join(results_dir,'dc_xgboost_folds_results.csv'))
feat_imps.to_csv(os.path.join(results_dir,'dc_xgboost_folds_feat_imps.csv'))
prob_preds.to_csv(os.path.join(results_dir,'dc_xgboost_folds_prob_peds.csv'))
cr_df.to_csv(os.path.join(results_dir,'dc_xgboost_classification_report_avg.csv'))